In [1]:
import cv2
from ultralytics import YOLO
from collections import defaultdict


In [2]:
model = YOLO("yolo11n.pt")

In [3]:
# Get the class labels from the model
class_list = model.names
print(class_list)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [7]:
# import cv2
# from ultralytics import YOLO
# from collections import defaultdict

# # Load the YOLO model
# model = YOLO("yolo11n.pt")

# # Get the class labels from the model
# class_list = model.names
# print("Class Labels:", class_list)

# Open the video file
cap = cv2.VideoCapture("video3.mp4")

# Line position for counting
line_y_red = 430  # Red line position

# Dictionary to store the count of each class
class_counts = defaultdict(int)

# Set to keep track of object IDs that have crossed the red line
crossed_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO on the frame
    results = model.track(frame, persist=True, classes=[0, 1, 2, 3, 5, 6, 7])

    # Ensuring results are not empty and boxes exist
    if results[0].boxes is not None and len(results[0].boxes) > 0:
        # Safely get detected boxes, class IDs, and track IDs
        boxes = results[0].boxes.xyxy
        track_ids = results[0].boxes.id
        class_indices = results[0].boxes.cls
        confidences = results[0].boxes.conf

        # Check if any of these attributes are None
        if (
            boxes is not None
            and track_ids is not None
            and class_indices is not None
            and confidences is not None
        ):
            # Convert to NumPy arrays for easier processing
            boxes = boxes.cpu().numpy()
            track_ids = track_ids.cpu().numpy()
            class_indices = class_indices.cpu().numpy().astype(int)
            confidences = confidences.cpu().numpy()

            # Draw the counting line
            # cv2.line(frame, (690, line_y_red), (1130, line_y_red), (0, 0, 255), 3)

            # Loop through each detected object
            for box, track_id, class_idx, conf in zip(
                boxes, track_ids, class_indices, confidences
            ):
                x1, y1, x2, y2 = map(int, box)
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2

                class_name = class_list[class_idx]  # Get the class label

                # Draw a circle and bounding box around the object
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(
                    frame,
                    f"Id: {int(track_id)} {class_name}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 0, 255),
                    2,
                )
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # Check if the object has crossed the red line
                if cy > line_y_red and track_id not in crossed_ids:
                    # Mark the object as crossed
                    crossed_ids.add(track_id)
                    class_counts[class_name] += 1

        # Display the counts on the frame
        y_offset = 30
        for class_name, count in class_counts.items():
            cv2.putText(
                frame,
                f"{class_name}: {count}",
                (50, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (0, 0, 255),
                2,
            )
            y_offset += 30

    # Display the frame
    cv2.imshow("Traffic Monitoring", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()



0: 384x640 2 cars, 57.0ms
Speed: 1.0ms preprocess, 57.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 3 trucks, 62.5ms
Speed: 1.6ms preprocess, 62.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 3 trucks, 78.9ms
Speed: 1.7ms preprocess, 78.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 3 trucks, 66.0ms
Speed: 1.2ms preprocess, 66.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 3 trucks, 63.8ms
Speed: 1.9ms preprocess, 63.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 3 trucks, 60.9ms
Speed: 1.1ms preprocess, 60.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 3 trucks, 59.3ms
Speed: 1.5ms preprocess, 59.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 3 trucks, 